### Plot results of Nd field experiments

In [4]:
import sys
import os
from os.path import join
sys.path.append('../')

import torch 
import numpy as np 
import torch.nn as nn
from tqdm import tqdm
from utils.jac_tests import check_jac_sym, check_jac_psd
import pandas as pd

from test_functions import PolyMax
from utils.make_model import *

gpu = 0
trials = 15

if torch.cuda.is_available():
    device = torch.device("cuda:"+str(gpu))
    print("GPU available. Using : " + torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU.")

def load_model(model_name, path, filename):
    model = make_model(model_name, dim)
    model.load_state_dict(torch.load(join(path, filename)))
    return model

GPU available. Using : NVIDIA GeForce RTX 2080 Ti


In [5]:
def get_test_errs(fn_name, dim, folder, seed, device):
    results = []
    n_pts = 10000
    
    ## Load function
    field = PolyMax(dim, device)

    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    pts = torch.rand(size=(n_pts, dim)).to(device)
    true_grad = field(pts, train=False)
    for model_name in os.listdir(join(folder, fn_name, 'dim'+str(dim))):
        for filename in os.listdir(join(folder, fn_name, 'dim'+str(dim), model_name)):
            net = load_model(model_name, join(folder, fn_name, 'dim'+str(dim), model_name), filename)
            net.to(device)
            params = sum(p.size().numel() for p in net.parameters() if p.requires_grad)
            net.eval()

            with torch.no_grad():
                pred_grad = net(pts)
            
            err = torch.mean(torch.linalg.vector_norm(pred_grad - true_grad, dim=-1, ord=2)**2)
            err = 10 * torch.log10(err)
            results.append([model_name, err.item(), params, filename])
    df = pd.DataFrame(results, columns=['model', 'mse', 'params', 'filename'])
    return df

In [6]:
dims = [256, 1024]

for dim in dims:
    df = []
    for trial in range(trials):
        df_ = get_test_errs('polymax', dim, 'trained_models', trial + 10, device)
        df_['trial'] = trial
        df.append(df_)
    
    df = pd.concat(df, axis=0, ignore_index=True)
    
    print(f'\nPOLYMAX RESULTS : dim = {dim}')
    
    polymax_df = df.groupby(['model', 'filename', 'params'])['mse'].agg(['mean', 'std'])
    idx = polymax_df.groupby('model')['mean'].idxmin()
    polymax_df = polymax_df.loc[idx].reset_index()
    
    print(polymax_df)



POLYMAX RESULTS : dim = 256
   model           filename  params       mean       std
0  mGN-C   model-lr0.001.pt  261904 -10.637234  0.014446
1  mGN-M  model-lr0.0001.pt  261384 -11.297316  0.015212

POLYMAX RESULTS : dim = 1024
   model           filename   params       mean       std
0  mGN-C  model-lr0.0001.pt  1048156  -8.747474  0.019742
1  mGN-M   model-lr0.001.pt  1046540 -10.419901  0.012317
